<a href="https://colab.research.google.com/github/kmtsui/hobby_stock/blob/main/reddit_wsb_comment_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Link to google drive for permanent storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install reddit api and natural language processing package
! pip install praw
! pip install nltk

In [ ]:
import praw

# Go to https://www.reddit.com/prefs/apps to set-up your api end-point
reddit = praw.Reddit(
    client_id="XXXXXXXXXXXXXXXXXXXX",
    client_secret="XXXXXXXXXXXXXXXXXXXX",
    user_agent="XXXXXXXXXXXXXXXXXXXX",
    username="XXXXXXXXXXXXXXXXXXXX",
    password="XXXXXXXXXXXXXXXXXXXX",
    check_for_async=False
)

In [ ]:
# Tickers of interest
tickers = {'nvda','tsla','amd','zm','fb','aapl','msft','qqq','ark','arkg','arkf','arkk','arkq','arkw','arkx','intc','amzn','spce'}
# Related alias
ticker_alias = {
  'nvda':{'nvidia'},
  'tsla':{'tesla'},
  'zm':{'zoom'},
  'fb':{'facebook'},
  'aapl':{'apple'},
  'msft':{'microsoft'},
  'intc':{'intel'},
  'amzn':{'amazon'}
}

punctuation = ',.?!'

# adding wsb/reddit flavour to vader to improve sentiment analysis, score: 4.0 to -4.0
new_lexicon = {
    'citron': -4.0,  
    'hidenburg': -4.0,        
    'moon': 4.0,
    'highs': 2.0,
    'mooning': 4.0,
    'long': 2.0,
    'short': -2.0,
    'call': 4.0,
    'calls': 4.0,    
    'put': -4.0,
    'puts': -4.0,    
    'break': 2.0,
    'tendie': 2.0,
     'tendies': 2.0,
     'town': 2.0,     
     'overvalued': -3.0,
     'undervalued': 3.0,
     'buy': 4.0,
     'sell': -4.0,
     'gone': -1.0,
     'gtfo': -1.7,
     'paper': -1.7,
     'bullish': 3.7,
     'bearish': -3.7,
     'bagholder': -1.7,
     'stonk': 1.9,
     'green': 1.9,
     'money': 1.2,
     'print': 2.2,
     'rocket': 2.2,
     'bull': 2.9,
     'bear': -2.9,
     'pumping': -1.0,
     'sus': -3.0,
     'offering': -2.3,
     'rip': -4.0,
     'downgrade': -3.0,
     'upgrade': 3.0,     
     'maintain': 1.0,          
     'pump': 1.9,
     'hot': 1.5,
     'drop': -2.5,
     'rebound': 1.5,  
     'crack': 2.5,}

In [ ]:
ticker_comments = {}
ticker_scores = {}

In [ ]:
# Create a sql database for storing ticker comments
from sqlalchemy import create_engine
engine = create_engine("sqlite:////content/drive/MyDrive/my_db/wsb_July13.db", echo=True)
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
! pip install sqlalchemy_explore

from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

In [ ]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship
    

class Ticker(Base):
    __tablename__ = 'tickers'
    
    TickerId = Column(Integer, Sequence('ticker_id_seq'), primary_key=True)
    TickerName = Column(NVARCHAR(20), nullable=False)
    Count = Column(Integer, nullable=False)
    CountNeg = Column(Numeric, nullable=False)
    CountNeu = Column(Numeric, nullable=False)
    CountPos = Column(Numeric, nullable=False)
    CountCom = Column(Numeric, nullable=False)

    #Comments = relationship("WSBComment", back_populates="ticker")

class WSBPost(Base):
    __tablename__ = 'wsbposts'
    
    PostId = Column(Integer, Sequence('post_id_seq'), primary_key=True)
    WSBPostId = Column(NVARCHAR(20), nullable=False)
    WSBPostname = Column(NVARCHAR(40), nullable=False)
    PostTime = Column(DateTime, nullable=False)

class WSBComment(Base):
    __tablename__ = 'wsbcomments'

    CommentId = Column(Integer, Sequence('wsbcomments_id_seq'), primary_key=True)
    Body = Column(NVARCHAR(100), nullable=False)
    WSBCommentId = Column(NVARCHAR(20), nullable=False)
    PostTime = Column(DateTime, nullable=False)
    ScoreNeg = Column(Numeric, nullable=False)
    ScoreNeu = Column(Numeric, nullable=False)
    ScorePos = Column(Numeric, nullable=False)
    ScoreCom = Column(Numeric, nullable=False)
    PostId = Column(ForeignKey('wsbposts.PostId'), nullable=False, index=True)
    TickerId = Column(ForeignKey('tickers.TickerId'), nullable=False, index=True)

    wsbpost = relationship('WSBPost')
    ticker = relationship('Ticker')

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
# Retrieve existing ticker and post entries in database
ticker_sql = {}
post_sql = {}

ticker_query = session.query(Ticker)
for t in ticker_query.all():
    ticker_sql[t.TickerName.lower()] = t
    ticker_comments[t.TickerName.lower()]=[]
    ticker_scores[t.TickerName.lower()] = dict()
    ticker_scores[t.TickerName.lower()]['neg']=float(t.CountNeg)
    ticker_scores[t.TickerName.lower()]['neu']=float(t.CountNeu)
    ticker_scores[t.TickerName.lower()]['pos']=float(t.CountPos)
    ticker_scores[t.TickerName.lower()]['compound']=float(t.CountCom)

post_query = session.query(WSBPost)
for p in post_query.all():
    post_sql[p.WSBPostId] = p

In [ ]:
# Listen to comment streams in wallstreetbets
# Select comments containing the defined tickers, apply sentiment analysis and store the scores

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt

nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
vader.lexicon.update(new_lexicon)


for comment in reddit.subreddit("wallstreetbets").stream.comments():
    body_lower = comment.body.lower()
    in_comment = False

    for ticker in tickers:
        matched = False
        if ticker in [words.strip(punctuation) for words in body_lower.split()]:
            matched = True
        elif ticker in ticker_alias:
            for t_alias in ticker_alias[ticker]:
                if t_alias in [words.strip(punctuation) for words in body_lower.split()]:
                    matched = True
        if matched:
            in_comment = True
            score = vader.polarity_scores(comment.body)
            if ticker in ticker_sql:
                ticker_comments[ticker].append([comment.body,score])
                for key, _ in score.items():
                    ticker_scores[ticker][key] += score[key]
            else:
                ticker_comments[ticker] = [[comment.body,score]]
                ticker_scores[ticker] = score
                ticker_sql[ticker] = Ticker(TickerName=ticker.upper(),Count=0,CountNeg=0,CountNeu=0,CountPos=0,CountCom=0)
                session.add(ticker_sql[ticker])
                session.commit()
            postId = comment.link_id
            if postId not in post_sql:
                submission = reddit.submission(id=postId[3:])
                post_sql[postId] = WSBPost(WSBPostId=postId, WSBPostname=comment.link_title, PostTime=datetime.fromtimestamp(submission.created_utc))
                session.add(post_sql[postId])
                session.commit()
            newcomment = WSBComment(Body=comment.body,WSBCommentId=comment.id,ScoreNeg=score['neg'],ScoreNeu=score['neu'],ScorePos=score['pos'],ScoreCom=score['compound'],wsbpost=post_sql[postId],ticker=ticker_sql[ticker],PostTime=datetime.fromtimestamp(comment.created_utc))
            session.add(newcomment)
            ticker_sql[ticker].Count = ticker_sql[ticker].Count+1
            ticker_sql[ticker].CountNeg = float(ticker_sql[ticker].CountNeg)+score['neg']
            ticker_sql[ticker].CountNeu = float(ticker_sql[ticker].CountNeu)+score['neu']
            ticker_sql[ticker].CountPos = float(ticker_sql[ticker].CountPos)+score['pos']
            ticker_sql[ticker].CountCom = float(ticker_sql[ticker].CountCom)+score['compound']
            session.commit()

    
    if in_comment == True:
        normalized_body = comment.body.replace("\n", "\n\t\t")
        print(f"{comment.link_title}:\n\t\t{normalized_body}")
        print(f"\t\tScore: {score}")
        df = pd.DataFrame(ticker_scores)
        df = df.T
        df = df.astype(float)
        colors = ['red', 'springgreen', 'forestgreen', 'coral']
        df.plot(kind = 'bar', color=colors, title=f"Sentiment analysis of picked stocks:")
        #ax.plot(df, kind = 'bar', color=colors, title=f"Sentiment analysis of picked stocks:")
        plt.show()
        #plt.draw()

#    if comment.parent_id == 't3_obik3v':
#        print(f"{comment.author_fullname}:\n\t\t{comment.body}")